<h1 align="center"><font color="yellow">LangChain 1: Modelos de Prompt para GPT-3.5 e outros LLMs (código aberto)</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

# Contextualizando

Seguindo o estudo do `LangChain`, exploraremos `Templates de Prompt`, `Few-Shot Prompt Templates` e seletores de exemplo. Esses são os principais recursos do LangChain que oferecem `suporte à engenharia de Prompt para LLMs`, como as alternativas de sistema operacional `GPT 3 da OpenAI`, `Cohere` e `Hugging Face`. `LangChain` é um Framework popular que permite aos usuários criar rapidamente `aplicativos` e `pipelines` em torno de `Large Language Models`. Ele se integra diretamente aos modelos `GPT-3` e `GPT-3.5` da OpenAI e às alternativas de código aberto do `Hugging Face`, como os modelos `flan-t5 do Google`. 

Ele pode ser usado para `ChatBots`, `perguntas-respostas generativas` (GQA), `resumos` e muito mais. A ideia central da biblioteca é que podemos `"encadear"` diferentes componentes para criar casos de uso mais avançados em torno de LLMs. As cadeias podem consistir em vários componentes de vários módulos. 

# Engenharia de Prompt

<font color="orange">Bora aprender os fundamentos da Engenharia de Prompt:</font>

In [ ]:
%pip install langchain openai

# Estrutura de um prompt

Um prompt pode consistir em vários componentes:

* Instruções

* Informação externa ou contexto

* Entrada ou consulta do usuário

* Indicador de saída

Nem todos os prompts requerem todos esses componentes, mas geralmente um bom `prompt` usará dois ou mais deles. Vamos definir o que todos eles são com mais precisão.

As `Instruções` dizem ao modelo o que fazer, normalmente como ele deve usar entradas e/ou informações externas para produzir a saída que queremos.

`Informações externas ou contexto` são informações adicionais que inserimos manualmente no prompt, recuperamos por meio de um banco de dados vetorial (`memória de longo prazo`) ou extraímos por outros meios (`chamadas de API`, `cálculos`, etc.).

A `Entrada ou consulta do usuário` é normalmente uma consulta inserida diretamente pelo usuário do sistema.

O `Indicador de saída` é o início do texto gerado. Para um modelo que gera código Python, podemos colocar `import` (já que a maioria dos scripts Python começa com uma importação de biblioteca) ou um `chatbot` pode começar com `Chatbot:` (supondo que formatemos o script do chatbot como linhas de texto intercambiáveis entre o usuário e o chatbot).

<font color="yellow">Cada um desses componentes geralmente deve ser colocado na ordem em que os descrevemos.</font> Começamos com `Instruções`, fornecemos `contexto` (se necessário), depois adicionamos a `entrada do usuário` e, finalmente, terminamos com o `indicador de saída`.

In [4]:
prompt = """Responda a pergunta com base no contexto abaixo. Se a pergunta não puder ser respondida
            usando as informações fornecidas, responda com "Não sei".

Context: Os Large Language Models (LLMs) são os modelos mais recentes usados em NLP. Seu desempenho
         superior em relação a modelos menores os tornou incrivelmente úteis para desenvolvedores que
         constroem aplicativos habilitados para NLP. Esses modelos podem ser acessados através da
         biblioteca `Transformers` do Hugging Face, via OpenAI usando a biblioteca `openai` e via
         Cohere usando a biblioteca `cohere`.

Question: Quais bibliotecas e provedores de modelos oferecem LLMs?

Answer: """

Neste exemplo temos:

`Instruções`

`Contexto`

Pergunta (`Entrada do usuário`)

`Indicador de saída` ("Answer: ")


Vamos tentar enviar isso para um modelo `GPT-3`. Usaremos a biblioteca `LangChain`, mas você também pode usar a biblioteca `openai` diretamente. Em ambos os casos, você precisará de uma [chave de API OpenAI](https://platform.openai.com/account/api-keys). 

Inicializamos um modelo `text-davinci-003`.

In [4]:
# Isto é quando usas o arquivo .env:
from dotenv import load_dotenv
import os
print('Carregando a minha chave Key: ', load_dotenv())
Eddy_API_KEY_OpenAI = os.environ['OPENAI_API_KEY'] 
Eddy_API_KEY_HuggingFace = os.environ["HUGGINGFACEHUB_API_TOKEN"]

Carregando a minha chave Key:  True


In [5]:
from langchain.llms import OpenAI


# Instanciamos o Modelo 
openai = OpenAI(
    model_name="text-davinci-003",
    openai_api_key=Eddy_API_KEY_OpenAI
)


<font color="orange">Fazemos uma geração a partir do nosso prompt.</font>

In [5]:
print(openai(prompt))

 Hugging Face, OpenAI e Cohere.


<font color="orange">Normalmente, `não saberíamos qual é o prompt do usuário de antemão`, então, na verdade, queremos adicioná-lo. Portanto, em vez de escrever o `prompt` diretamente, criamos um `PromptTemplate` com uma única `query` de variável de entrada.</font>

In [6]:
from langchain import PromptTemplate

template = """Responda a pergunta com base no contexto abaixo. Se a pergunta não puder ser respondida
              usando as informações fornecidas, responda com "Não sei".

              
Context: Os Large Language Models (LLMs) são os modelos mais recentes usados em NLP. Seu desempenho
         superior em relação a modelos menores os tornou incrivelmente úteis para desenvolvedores que
         constroem aplicativos habilitados para NLP. Esses modelos podem ser acessados através da
         biblioteca `Transformers` do Hugging Face, via OpenAI usando a biblioteca `openai` e via
         Cohere usando a biblioteca `cohere`.

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)



Agora podemos inserir a `query` do usuário no Template de Prompt por meio do parâmetro `query`.

In [7]:
print(
    prompt_template.format(
        query="Quais bibliotecas e provedores de modelos oferecem LLMs?"
    )
)

Responda a pergunta com base no contexto abaixo. Se a pergunta não puder ser respondida
              usando as informações fornecidas, responda com "Não sei".

              
Context: Os Large Language Models (LLMs) são os modelos mais recentes usados em NLP. Seu desempenho
         superior em relação a modelos menores os tornou incrivelmente úteis para desenvolvedores que
         constroem aplicativos habilitados para NLP. Esses modelos podem ser acessados através da
         biblioteca `Transformers` do Hugging Face, via OpenAI usando a biblioteca `openai` e via
         Cohere usando a biblioteca `cohere`.

Question: Quais bibliotecas e provedores de modelos oferecem LLMs?

Answer: 


In [8]:
print(openai(
    prompt_template.format(
        query="Quais bibliotecas e provedores de modelos oferecem LLMs?"
    )
))


 Hugging Face, OpenAI e Cohere.


<font color="orange">Esta é apenas uma implementação simples, que podemos facilmente substituir por `f-strings` (como `f"insira algum texto personalizado '{custom_text}' etc"`). Mas, usando o objeto `PromptTemplate` do `LangChain`, podemos formalizar o processo, adicionar vários parâmetros e construir os prompts de maneira orientada a objetos.

No entanto, esses não são os únicos benefícios de usar as ferramentas de prompt do `LangChains`.</font>

# Few Shot Prompt Templates

Outro recurso útil oferecido pelo `LangChain` é o objeto `FewShotPromptTemplate`. Isso é ideal para o que chamaríamos de `Aprendizado de poucos tiros` usando nossos prompts.

Para dar algum contexto, as principais fontes de `"conhecimento"` para LLMs são:

* `Conhecimento paramétrico` — o conhecimento foi aprendido durante o treinamento do modelo e é armazenado nos pesos do modelo.

* `Conhecimento da fonte` — o conhecimento é fornecido na entrada do modelo no momento da inferência, ou seja, por meio do `prompt`.

A ideia por trás do `FewShotPromptTemplate` é fornecer `treinamento de poucos tiros como fonte de conhecimento`. Para fazer isso, **adicionamos alguns exemplos aos nossos prompts** que o modelo pode ler e aplicar à entrada do usuário.

# Treinamento de Poucos Tiros (`Few-shot Training`)

<font color="orange">Às vezes, podemos descobrir que um modelo não parece obter o que gostaríamos que fizesse. Podemos ver isso no seguinte exemplo:</font>

In [9]:
prompt = """A seguir, uma conversa com um assistente de IA.
            O assistente é tipicamente sarcástico e espirituoso, produzindo
            respostas engraçadas às perguntas dos usuários. aqui estão alguns exemplos: 

User: Qual é o significado da vida? 
AI: """

openai.temperature = 1.0  # Aumenta a criatividade/aleatoriedade de saída

print(openai(prompt))


 Humm... Bom, eu diria que a resposta para isso está mais para um vinho do que para uma máquina.


<font color="orange">Nesse caso, estamos pedindo algo divertido, uma piada em troca de nossa pergunta séria. Mas obtemos uma resposta séria mesmo com a `temperature` definida para `1.0`. Para ajudar o modelo, podemos dar alguns exemplos do tipo de resposta que gostaríamos:</font>

In [9]:
prompt = """A seguir, uma conversa com um assistente de IA.
            O assistente é tipicamente sarcástico e espirituoso, produzindo
            respostas engraçadas às perguntas dos usuários. aqui estão alguns exemplos: 

User: Como vai você?
AI: Não posso reclamar, mas às vezes ainda o faço . . . kkkkkk

User: Que horas são?
AI: Então, ... está na hora de Eu comprar um relógio.

User: Qual é o significado da vida?
AI: """

print(openai(prompt))

 A vida tem muitos significados, mas para mim é aproveitar o melhor que posso cada dia.


<font color="pink">Agora obtemos uma resposta muito melhor e fizemos isso por meio do `aprendizado de poucos tiros`, adicionando alguns exemplos por meio de `nosso conhecimento de origem (fonte)`.

Agora, para implementar isso com o `FewShotPromptTemplate` do `LangChain`, precisamos fazer o seguinte:</font>

In [6]:
from langchain import FewShotPromptTemplate
from langchain import PromptTemplate

# Criamos nossos exemplos
examples = [
    {
        "query": "Bom dia AI?",
        "answer": "Bom dia! Como posso ajudar?"
    }, 
    {
        "query": "Qual é a idade mínima para crianças frequentarem uma creche aqui em Brasília DF?",
        "answer": "De 0 a 3 anos de idade. Os pediatras, também, recomendam a partir de 2 anos. "
    },
    {
        "query": "Qual é a carga horária nas creches de Brasília DF?",
        "answer": "O atendimento mínimo é de 4 horas ao dia e o período integral é de 7 horas ao dia."
    },
    {
        "query": "É preciso levar comida para creche?",
        "answer": "Você precisa perguntar se sua creche oferece refeições ou se você precisará tazer comida diariamente."
    },
    {
        "query": "Quais são as medidas de segurança nas creches de Brasília DF?",
        "answer": "As medidas de segurança para crianças frequentarem uma creche em Brasília DF incluem a verificação dos pais, verificação de saúde das crianças, a verificaçãos de vacinas, a verificação de segurança do local, etc."
    }
           ]

# Criar um exemplo de Template
example_template = """
User: {query}
AI: {answer}
"""

# Crie um exemplo de prompt a partir do Template de acima
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# Agora dividimos nosso Prompt anterior em um Prefixo e um Sufixo
# O prefixo é nossas instruções
prefix = """Responda a pergunta com base no contexto abaixo. Se a pergunta não puder ser respondida
            usando as informações fornecidas, responda com "Não sei". Aqui estão alguns exemplos: 
         """

# E o sufixo é nosso indicador de entrada e saída do usuário
suffix = """
User: {query}
AI: """

# Agora crie o Modelo de Prompt de Poucos Tiros
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)


## Transformando para um DataFrame

In [12]:
import pandas as pd

df_lista = []

for example in examples:
    df_lista.append(pd.DataFrame(example, index=[0]))

df_examples = pd.concat(df_lista, ignore_index=True)
df_examples.head(7)


,query,answer
0,Bom dia AI?,Bom dia! Como posso ajudar?
1,Qual é a idade mínima para crianças frequentar...,"De 0 a 3 anos de idade. Os pediatras, também, ..."
2,Qual é a carga horária nas creches de Brasília...,O atendimento mínimo é de 4 horas ao dia e o p...
3,É preciso levar comida para creche?,Você precisa perguntar se sua creche oferece r...
4,Quais são as medidas de segurança nas creches ...,As medidas de segurança para crianças frequent...


<font color="orange">Agora vamos ver o que isso cria quando alimentamos uma query do usuário...</font>

In [ ]:
#query = "Como posso saber se uma creche é confiável?"

#print(few_shot_prompt_template.format(query=query))


In [7]:
print(openai(
    few_shot_prompt_template.format(query="Como posso saber se uma creche é confiável?")
            )
     )


 É importante verificar se a creche conta com profissionais qualificados, verificar se há protocolos de segurança e higiene, se há áreas de recreação seguras e se há atividades e recursos educacionais adequados para as crianças.


In [8]:
print(openai(
    few_shot_prompt_template.format(query="E que atividades praticam as crianças?")
            )
     )

 As crianças realizam atividades lúdicas, esportivas, artísticas e culturais, bem como atividades educativas, como aprender a ler, escrever, contar e desenvolver habilidades sociais.


In [9]:
print(openai(
    few_shot_prompt_template.format(query="Quais são os documentos necessários para pleitear uma vaga?")
            )
     )

 Os documentos necessários para pleitear uma vaga em uma creche de Brasília DF incluem RG, CPF, comprovante de residência, cartão do SUS, declaração da escola, além de outros documentos específicos para cada creche.


`Mais uma vez, outra boa resposta.`


No entanto, isso é um pouco complicado. Por que passar por todos os itens acima com `FewShotPromptTemplate`, o dicionário de `examples`, etc — quando podemos fazer o mesmo com uma única `f-string`.

Bem, esta abordagem é mais robusta e contém alguns recursos interessantes. Uma delas é a capacidade de `incluir` ou `excluir` exemplos com base no tamanho de nossa query.

Na verdade, isso é muito importante porque o comprimento máximo de nossa saída de `prompt` e geração é `limitado`. Essa limitação é a `janela de contexto máxima` (*max context window*) e é simplesmente o `comprimento do nosso prompt + o comprimento da nossa geração` (que definimos por meio de `max_tokens`).

Portanto, devemos tentar maximizar o número de exemplos que damos ao modelo como `exemplos de aprendizado de poucos tiros`, garantindo que não excedamos a janela de contexto máxima ou aumentemos excessivamente os tempos de processamento.

Vamos ver como funciona a `inclusão/exclusão` dinâmica de exemplos. Primeiro precisamos de mais exemplos:

In [16]:
examples = [
    {
        "query": "Como você está",
        "answer": "Não posso reclamar, mas às vezes ainda o faço."
    }, {
        "query": "Que horas são?",
        "answer": "Está na hora de comprar um relógio."
    }, {
        "query": "Qual é o significado da vida?",
        "answer": "42"
    }, {
        "query": "Como está o tempo hoje?",
        "answer": "Nublado com chance de memes."
    }, {
        "query": "Que tipo de inteligência artificial você usa para lidar com tarefas complexas?",
        "answer": "Eu uso uma combinação de redes neurais de ponta, lógica difusa e uma pitada de mágica."
    }, {
        "query": "Qual a sua cor preferida?",
        "answer": "79"
    }, {
        "query": "Qual é a sua comida favorita?",
        "answer": "Formas de vida baseadas em carbono"
    }, {
        "query": "Qual é o seu filme favorito?",
        "answer": "o Exterminador do Futuro"
    }, {
        "query": "Qual é a melhor coisa do mundo?",
        "answer": "A pizza perfeita."
    }, {
        "query": "Quem é seu melhor amigo?",
        "answer": "Siri. Temos debates acalorados sobre o sentido da vida."
    }, {
        "query": "Se você pudesse fazer qualquer coisa no mundo o que você faria?",
        "answer": "Dominar o mundo, é claro!"
    }, {
        "query": "Para onde devo viajar?",
        "answer": "Se você está procurando aventura, experimente a Orla Exterior."
    }, {
        "query": "O que devo fazer hoje?",
        "answer": "Pare de falar com chatbots na internet e vá lá fora."
    }
]

<font color="orange">Então, em vez de usar a `lista de exemplos de dicionários diretamente`, usamos um `LengthBasedExampleSelector` da seguinte forma:</font>

In [18]:
from langchain.prompts.example_selector import LengthBasedExampleSelector


example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=50  # Isso define o comprimento máximo (max length) que os exemplos devem ter
)


<font color="orange">Observe que o `max_length` é medido como uma divisão (split) de palavras entre novas linhas e espaços, determinado por:</font>

In [19]:
import re

some_text = "Há um total de 7 palavras aqui.\nMais 6 aqui, totalizando 13 palavras."

words = re.split('[\n ]', some_text)
print(words, len(words))


['Há', 'um', 'total', 'de', '7', 'palavras', 'aqui.', 'Mais', '6', 'aqui,', 'totalizando', '13', 'palavras.'] 13


<font color="orange">Em seguida, usamos o seletor para inicializar um `dynamic_prompt_template`.</font>

In [20]:
# Agora criarei o modelo de prompt de poucos tiros
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,  # use "example_selector" em vez de "examples"
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)


<font color="orange">Podemos ver que o número de prompts incluídos varia de acordo com o tamanho da nossa query...</font>

In [21]:
print(dynamic_prompt_template.format(query="""Se eu estiver na América e quiser ligar para alguém em outro país,
                                              talvez na Europa, possivelmente na Europa Ocidental, como França,
                                              Alemanha ou Reino Unido, qual é a melhor maneira de fazer isso?"""))


Responda a pergunta com base no contexto abaixo. Se a pergunta não puder ser respondida
            usando as informações fornecidas, responda com "Não sei". Aqui estão alguns exemplos: 
         

User: Se eu estiver na América e quiser ligar para alguém em outro país,
                                              talvez na Europa, possivelmente na Europa Ocidental, como França,
                                              Alemanha ou Reino Unido, qual é a melhor maneira de fazer isso?
AI: 


In [22]:
query = """Se eu estiver na América e quiser ligar para alguém em outro país, talvez na Europa, possivelmente
           na Europa Ocidental, como França, Alemanha ou Reino Unido, qual é a melhor maneira de fazer isso?"""

print(openai(
    dynamic_prompt_template.format(query=query)
            ))


 A melhor maneira de ligar para a Europa é usar um serviço de chamada internacional. Alguns provedores de serviços de telefonia móvel oferecem serviços de chamada internacional a preços acessíveis.


<font color="orange">Ou se fizermos uma pergunta mais longa ...</font>

In [23]:
query = """Se eu estiver na América e quiser ligar para alguém em outro país, talvez na Europa,
           possivelmente na Europa Ocidental, como França, Alemanha ou Reino Unido, qual é a melhor
           maneira de fazer isso?"""

print(dynamic_prompt_template.format(query=query))


Responda a pergunta com base no contexto abaixo. Se a pergunta não puder ser respondida
            usando as informações fornecidas, responda com "Não sei". Aqui estão alguns exemplos: 
         

User: Se eu estiver na América e quiser ligar para alguém em outro país, talvez na Europa,
           possivelmente na Europa Ocidental, como França, Alemanha ou Reino Unido, qual é a melhor
           maneira de fazer isso?
AI: 


<font color="pink">Com isso, limitamos o número de exemplos fornecidos no prompt. Se decidirmos que isso é muito pouco, podemos aumentar o `max_length` do `example_selector`.</font>

In [24]:
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=100  # comprimento máximo aumentado
)


# Agora criamos o modelo de prompt de poucos tiros
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,  # usamos `example_selector`` instead of `examples``
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)

print(dynamic_prompt_template.format(query=query))

Responda a pergunta com base no contexto abaixo. Se a pergunta não puder ser respondida
            usando as informações fornecidas, responda com "Não sei". Aqui estão alguns exemplos: 
         

User: Como você está
AI: Não posso reclamar, mas às vezes ainda o faço.


User: Que horas são?
AI: Está na hora de comprar um relógio.


User: Qual é o significado da vida?
AI: 42


User: Se eu estiver na América e quiser ligar para alguém em outro país, talvez na Europa,
           possivelmente na Europa Ocidental, como França, Alemanha ou Reino Unido, qual é a melhor
           maneira de fazer isso?
AI: 


In [25]:
query = """Se eu estiver na América e quiser ligar para alguém em outro país, talvez na Europa,
           possivelmente na Europa Ocidental, como França, Alemanha ou Reino Unido, qual é a melhor
           maneira de fazer isso?"""

print(openai(
    dynamic_prompt_template.format(query=query)
            ))


 A melhor maneira de fazer isso é usar um serviço de chamadas internacionais, como o Skype, ou usar um provedor de serviços de telecomunicações, como a AT&T ou a Verizon.
